In [47]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [49]:
download = drive.CreateFile({'id': '1Hydbg7KXOoK9Xf-DPFsoMGv6Whdzhh1F'})
download.GetContentFile('train_full.csv')
download = drive.CreateFile({'id': '1uo-5rbDJKUSBondu2NnaMC4ygu1TwuW0'})
download.GetContentFile('test_full.csv')

2019-04-19 07:00:49,945 : WARNING : file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No mo

In [0]:
import pandas as pd
import numpy as np

label_name = 'insult'

train_full = pd.read_csv('train_full.csv')
test_full = pd.read_csv('test_full.csv')



In [51]:
#####INSULT

test1 = test_full[ test_full[label_name]==1 ]
test0 = test_full[ test_full[label_name]==0 ]

train1 = train_full[ train_full[label_name]==1 ]
train0 = train_full[ train_full[label_name]==0 ]

test0_new = test0.iloc[0:3000,:]
train0_new = train0.iloc[0:5000,:]


train_full = pd.concat([train1,train0_new],axis='rows',ignore_index=True).reset_index(drop=True)
test_full = pd.concat([test1,test0_new],axis='rows',ignore_index=True).reset_index(drop=True)

#shuffling concatenated dataframe:
train_full = train_full.sample(frac=1).reset_index(drop=True)
test_full = test_full.sample(frac=1).reset_index(drop=True)

print(train_full.shape,test_full.shape,list(train_full))


(9864, 9) (5426, 9) ['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'non-toxic', 'comment_text']


Create word vectors

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report

In [0]:
train = train_full["comment_text"]
test = test_full["comment_text"]

trainlab = train_full[label_name]
testlab = test_full[label_name]

In [54]:
# string cleaning- already done
# Download and load the punkt tokenizer for sentence splitting
import nltk.data
nltk.download("popular")   
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [0]:
# to split a review into parsed sentences returning a list of sentences (each sentence is list of words)
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    review = str(review)
    raw_sentences = tokenizer.tokenize(review.strip()) # split para into sentences
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            #sentences.append( review_to_wordlist(raw_sentence,remove_stopwords) )
            sentences.append(raw_sentence) #bcz already cleaned h
    return sentences

In [56]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for r in train:
  sentences += review_to_sentences(r, tokenizer)

print ("Parsing sentences from testing set")
for r in test:
    sentences += review_to_sentences(r, tokenizer)
    
print(sentences[0:5])

Parsing sentences from training set
Parsing sentences from testing set
['onorem still faeggot', 'clue bot ng sucked cock inches huge penis like fuck icelantic girls ass', 'coming back kick teeth enough enough fatuorum', 'artsy fancy language thing took introduction art portal', 'brandon semenuk event four times idiot look trying make good page stop changing']


In [0]:
#training and saving model 
import logging # for nice output messages
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [58]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
div_factor = 1000    # division factor for clusters
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

2019-04-19 07:00:54,401 : INFO : collecting all words and their counts
2019-04-19 07:00:54,403 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2019-04-19 07:00:54,406 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model...


2019-04-19 07:00:54,804 : INFO : PROGRESS: at sentence #10000, processed 2079715 words, keeping 27 word types
2019-04-19 07:00:55,000 : INFO : collected 27 word types from a corpus of 3150444 raw words and 15290 sentences
2019-04-19 07:00:55,002 : INFO : Loading a fresh vocabulary
2019-04-19 07:00:55,003 : INFO : effective_min_count=40 retains 27 unique words (100% of original 27, drops 0)
2019-04-19 07:00:55,005 : INFO : effective_min_count=40 leaves 3150444 word corpus (100% of original 3150444, drops 0)
2019-04-19 07:00:55,006 : INFO : deleting the raw counts dictionary of 27 items
2019-04-19 07:00:55,007 : INFO : sample=0.001 downsamples 23 most-common words
2019-04-19 07:00:55,008 : INFO : downsampling leaves estimated 543933 word corpus (17.3% of prior 3150444)
2019-04-19 07:00:55,010 : INFO : estimated required memory for 27 words and 300 dimensions: 78300 bytes
2019-04-19 07:00:55,014 : INFO : resetting layer weights
2019-04-19 07:00:55,017 : INFO : training model with 4 worker

Converting review to word vectors

In [0]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print ("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[int(counter)] = makeFeatureVec(review, model, num_features)
       print(reviewFeatureVecs)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [60]:
trainDataVecs = getAvgFeatureVecs( train, model, num_features )
testDataVecs = getAvgFeatureVecs( test, model, num_features )


Output hidden; open in https://colab.research.google.com to view.

Random Forest Classifier

In [61]:
##bullshit for the results - RFC
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(trainDataVecs,train_full[label_name])
pred = clf.predict(testDataVecs)
from sklearn.metrics import classification_report
#test data
print(classification_report(test_full[label_name], pred))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           0       0.69      0.75      0.72      3000
           1       0.65      0.58      0.61      2426

   micro avg       0.67      0.67      0.67      5426
   macro avg       0.67      0.66      0.67      5426
weighted avg       0.67      0.67      0.67      5426



SVM poly kernel

In [0]:
##bullshit for the results
from sklearn.svm import SVC
clf = SVC(kernel='poly')
clf.fit(trainDataVecs,train_full[label_name])
pred = clf.predict(testDataVecs)
from sklearn.metrics import classification_report
#test data
print(classification_report(test_full[label_name], pred))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

           0       0.55      1.00      0.71      3000
           1       0.00      0.00      0.00      2426

   micro avg       0.55      0.55      0.55      5426
   macro avg       0.28      0.50      0.36      5426
weighted avg       0.31      0.55      0.39      5426



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Clustering these vectors

In [0]:
type(trainDataVecs)
print(trainDataVecs.shape)
tot = len(trainDataVecs)+len(testDataVecs)
print(tot)

arr = np.zeros((tot, num_features))

for col in range(0,num_features):
  for row in range(0, len(trainDataVecs)):
    arr[row] = np.array(trainDataVecs[row])
  
  
print(arr.shape)
l1 = len(trainDataVecs)
l2 = len(testDataVecs)
  
  
for col in range(0,num_features):
  for row in range(0, len(testDataVecs)):
    arr[l1 + row] = np.array(testDataVecs[row])


#word_vectors = np.concatenate(trainDataVecs,testDataVecs)
#print(word_vectors.shape)


(9864, 300)
15290
(15290, 300)


In [0]:
word_vectors = np.matrix(arr)
print(word_vectors.shape)

(15290, 300)


In [0]:
#word_vectors = pd.concat([trainDataVecs,testDataVecs])
#num_clusters = int(len(word_vectors) / div_factor)
#from sklearn.cluster import DBSCAN


num_clusters = 10

# Initalize a k-means object and use it to extract centroids
import time

start = time.time() # Start time


clusterer = KMeans( n_clusters = num_clusters, max_iter=500,  n_init=20)

idx = clusterer.fit_predict( word_vectors )


# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")
print (idx)

Time taken for K Means clustering:  23.921093463897705 seconds.
[2 9 0 ... 3 2 3]


In [0]:
trainVecs = np.matrix(trainDataVecs)
testVecs = np.matrix(testDataVecs)

train_clus = idx[0:l1]
test_clus = idx[l1:l1+l2]

trainVecs = np.c_[trainVecs,train_clus]
testVecs = np.c_[testVecs,test_clus]
train_df = pd.DataFrame(trainVecs)
test_df = pd.DataFrame(testVecs)

tr_df = train_df.rename({num_features:'cluster'}, axis='columns')
test_df = test_df.rename({num_features:'cluster'}, axis='columns')
print(tr_df.shape,len(trainlab),list(tr_df))


tl = pd.DataFrame(data={label_name:trainlab}) #np.zeros([l1,])


#tl=pd.Series(train_full[label_name])

train_df = pd.concat([tr_df,trainlab],axis='columns') #adds extra column for labels


(9864, 301) 9864 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 21

Training models for each cluster

In [0]:
print(trainVecs.shape,testVecs.shape, list(train_df))

(9864, 301) (5426, 301) [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 

1. Using probability within cluster to determine toxicity

In [0]:
print(num_clusters)

10


In [0]:
cluster_labels = []
for clus in range(0,num_clusters):
  
  clus_set = train_df [train_df['cluster'] == clus]
  labels = clus_set[label_name]
  count1 = len(clus_set [clus_set[label_name] == 1] ) #counts number of non-toxic rows in cluster
  count0 = len(clus_set [clus_set[label_name] == 0] ) #counts number of toxic rows in cluster
  if(count0 > count1):
    probable_pred = 0
  else:
    probable_pred = 1
  cluster_labels.append(probable_pred)
  
  if(clus%100 == 0):
    print('Cluster no ',clus)
    print('Probable pred', probable_pred)

print(len(cluster_labels))

Cluster no  0
Probable pred 1
10


In [0]:
prediction = []
#print(test_df['cluster'])

for clus_id in test_df['cluster']:

  #print(clus_id)
  prediction.append( cluster_labels[ int(clus_id) ] )
  
  
print(len(prediction),len(test_df))

5426 5426


In [0]:
from sklearn.metrics import classification_report
#test data
print(classification_report(testlab, prediction))

              precision    recall  f1-score   support

           0       0.70      0.71      0.71      3000
           1       0.64      0.63      0.63      2426

   micro avg       0.67      0.67      0.67      5426
   macro avg       0.67      0.67      0.67      5426
weighted avg       0.67      0.67      0.67      5426



Distance-weight implementation

In [0]:
z = np.zeros([l2,1])
preds = pd.DataFrame(data={'preds':[z] })  
test_df = pd.concat([test_df,preds],axis=1) #adds extra column for labels


In [0]:
list(test_df)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [0]:
def getEuclideanDist(vec1,vec2):
  dis = 0  
  #print(len(vec1),len(vec2))
  v1 = np.array(vec1)
  v2 = np.array(vec2)

  #print('for euclidean, columns: ',cols)
  
  for field in range(0,num_features):
    dis = dis + np.square( (v1[field] - v2[field]) )
    
  dis = np.sqrt(dis)
  return dis

In [0]:
def compareDistances(clus,testVec):
  
  clus_test = test_df[ test_df['cluster'] == clus ]
  
  
  clus_set = train_df [train_df['cluster'] == clus]
  list(clus_set)
  
  
  zero_wale = clus_set[ clus_set[label_name] == 0 ] #takes all non-toxic = 0 within cluster
  ones_wale = clus_set[ clus_set[label_name] == 1 ] #takes all non-toxic = 1 within cluster
  
  vector0 = zero_wale.drop(columns = [label_name,'cluster'])
  vector1 = ones_wale.drop(columns = [label_name,'cluster'])

  v0_len = len(vector0[0][:])
  v1_len = len(vector1[0][:])
  #print(v0_len,v1_len)
  
  distance0 = 0
  distance1 = 0
  
  for index in range(0,v0_len):
    trainVec = vector0.iloc[index,:]
    #print(trainVec.shape,testVec.shape)
    dist = getEuclideanDist(trainVec,testVec)
    distance0 = distance0 + dist
  
  
  for index in range(0,v1_len):
    trainVec = vector1.iloc[index,:]
    #print(trainVec.shape,testVec.shape)
    dist = getEuclideanDist(trainVec,testVec)
    distance1 = distance1 + dist
    
  if(distance0 > distance1):
    pred = 0
  else:
    pred = 1
    
  return pred
  
 

Predicting by using models per cluster

In [0]:
prediction = []
for j in range(0,l2):
  cluster_id = test_clus[j]
  testVector = test_df.iloc[j,0:num_features]
  #print(list(testVector.shape))
  
  pred = compareDistances(cluster_id, testVector)
  #print(j)
  prediction.append(pred)


KeyboardInterrupt: ignored

In [0]:
print(prediction)

In [0]:
len(prediction)

In [0]:
from sklearn.metrics import classification_report
#test data
print(classification_report(testlab, prediction))